# Demo: refactored implementation of $\Omega$

This notebook gives an example of the new implementation of $\Omega$. The aim of the implementation has so far been to make as few alterations as possible to the existing codebase, while still enabling the user considerably more flexibility in the definition of $\Omega$. 

## Math Assumptions

The primary mathematical assumption behind the new implementation is that $\Omega(z)$ is always expressible as a function of the partition vector of $z$. This is the same assumption that we were using throughout our previous version. The aim of the new implementation is that special cases can be handled with significantly more peformant code. 

In [60]:
using Pkg; Pkg.activate(".")
using HypergraphModularity

using StatsBase

 Activating environment at `~/hypergraph_modularities_code/Project.toml`


In [61]:
n = 20
Z = rand(1:5, n)
ϑ = dropdims(ones(1,n) + rand(1,n), dims = 1)
μ = mean(ϑ)
kmax = 4;

## The `IntensityFunction` struct

Now we're ready to construct $\Omega$. In the new idiom, $\Omega$ is an `IntensityFunction`, which contains four fields. 
Let's look at two examples, which are both implemented in `src/omega.jl`. 

```julia
function partitionIntensityFunction(ω, kmax)
    range      = partitionsUpTo(kmax)
    P          = partitionize
    aggregator = identity
    return IntensityFunction(ω, P, range, aggregator)
end
```

This `IntensityFunction` corresponds to the highly general `partitionize`-based framework we were using previously. Here are its fields: 

<dl>
    <dt> ω </dt> <dd> A parameterized function whose valid inputs are the elements of `range`. </dd>
    <dt> <code>P</code> </dt> <dd> A function that maps group label vectors (i.e. subvectors of $Z$) to <i>feature</i> vectors. <code>partitionize()</code> is an example, as is the polyadic $\delta$-function. </dd>
    <dt> <code>range</code> </dt> <dd> The set of all possible values of <code>P</code>. Also assumed to be the domain of the function $\omega$. This nomenclature is perhaps confusing and may be revised.</dd>
    <dt> <code>aggregator</code> </dt> <dd> A function that maps partition vectors (of the kind returned by <code>partitionize()</code> to elements of <code>range</code>. This is included for technical purposes related to the calculation of the volume term in modularity. It would be desirable to deprecate it if possible.  </dd>
</dl>

So, the code above creates an intensity function that operates on subvectors of $Z$ by first `partitionize()`ing them and then applying the function $\omega$. No aggregation is needed.  

On the other hand, here's an alternative that corresponds to the all-or-nothing cut. In this case, the feature map $P$ computes two entries: whether or not all entries of $z$ are the same, and the length of $z$. The `range` gives all possible results (up to size `kmax`). The aggregator takes a partition vector and returns a feature. The specified function $\omega$ should then operate on the features. 

```julia
function allOrNothingIntensityFunction(ω, kmax)
    range      = [(1.0*x, y) for x = 0:1 for y = 1:kmax]
    P          = z->(all(z[1] .== z), length(z))
    aggregator = p->(length(p) == 1, sum(p))
    return IntensityFunction(ω, P, range, aggregator)
end
```
Ok, let's try it out: 



In [62]:
function ω(p, α)
    k = sum(p)
    return sum(p)/sum((p .* (1:length(p)).^α[k])) / n^(α[kmax+k]*k)
end

α = vcat(repeat([5.0], kmax), 0.2*(1:kmax))
Ω = partitionIntensityFunction(ω, kmax);

typeof(Ω)

IntensityFunction

In [63]:
H = sampleSBM(Z, ϑ, Ω;α=α, kmax=kmax, kmin = 1)

hypergraph
  N: Array{Int64}((20,)) [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
  E: Dict{Int64,Dict}
  D: Array{Int64}((20,)) [2, 3, 1, 3, 12, 6, 9, 4, 7, 5, 4, 4, 6, 5, 9, 5, 7, 6, 6, 6]


# TODO: check on inference and algorithms

# CONJECTURE: 

we have introduced a bug related to CutDiff_OneNode that is causing issues. Consider whether we should try to squash the bug or reimplement SuperNodeLouvain using more familiar functions such as cutDiff(). Major question will be whether this introduces additional expense (i.e. whether we gain from the penalty term used in the current code). 

I **think** that we do gain from storying the penalties -- should cut the number of calls to `Omega.P` by about half, which is certainly desirable. 




In [64]:
dataset = "contact-primary-school"
kmax_ = 6

H, Z = read_hypergraph_data(dataset,kmax_)

kmax = maximum(keys(H.E))
kmin = minimum(keys(H.E))

n = length(H.D)

# full, partition-based intensity function

function ω_1(p, α)
    k = sum(p)
    return sum(p)/sum((p .* (1:length(p)).^α[k])) / n^(α[kmax+k]*k)
end

Ω_1 = partitionIntensityFunction(ω_1, kmax);

# ---

# sum of exterior
function ω_2(p,α)
    k = p[2]
    δ = p[1]
    return ((1+δ)*n)^α[k] / (1 + α[k + kmax])
end

Ω_2 = sumOfExteriorDegreesIntensityFunction(ω_2, kmax);


# sum of exterior
function ω_3(p,α)
    k = p[2]
    δ = p[1]
    return ((1+δ)*n)^α[k] / (1 + α[k + kmax])
end

Ω_3 = allOrNothingIntensityFunction(ω_3, kmax);

# Quick Speed Tests

In [65]:
Z_dyadic = CliqueExpansionModularity(H);

In [66]:
α = zeros(2*kmax);

## General Partition-Based

In [71]:
α, f = learnParameters(H, Z_dyadic, Ω_1, α; xtol_abs = 1e-6)

([0.3764741351374923, 2.8672942397473213, 6.742407149806297, 7.679033661187109, 7.064696734014081, 8.531123130937832, 0.9781161291263542, 1.1851031377792722, 1.4071890853292668, 1.6029789394102092], 207914.80460689426)

In [73]:
@time Z = HypergraphModularity.HyperLouvain(H, kmax, Ω_1; α=α, verbose = true);
println(length(unique(Z)))

Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
  4.969337 seconds (66.33 M allocations: 3.100 GiB, 11.15% gc time)
10


## Sum of Exterior Degrees

In [74]:
α = zeros(2*kmax);
α, f = learnParameters(H, Z_dyadic, Ω_2, α; xtol_abs = 1e-6)

([-1.266392248818159, -1.6246953011341447, -4.3745463505560735, -5.059854264169689, -6.971771486786326, 0.9471269502379152, 2.235520789533781, -0.998478541132391, 2.5056612050099116, 3.774895300335604], 211088.95913010038)

In [76]:
@time Z = HypergraphModularity.HyperLouvain(H, kmax, Ω_2; α=α, verbose = true);
println(length(unique(Z)))

Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
  3.855183 seconds (54.83 M allocations: 2.112 GiB, 10.30% gc time)
10


## All-or-nothing

In [77]:
α = zeros(2*kmax);

In [80]:
α, f = learnParameters(H, Z_dyadic, Ω_3, α; xtol_abs = 1e-6)

([-4.0284622622899535, -0.4762067433236213, -2.4097327497940086, -5.463881071470425, -8.721697714581483, 0.04306426279380271, 8130.039346766027, 10120.01747534906, 220.6160675525968, 3.7401930527651253], 227287.514631432)

In [85]:
using Profile

@profile Z = HypergraphModularity.HyperLouvain(H, kmax, Ω_3; α=α, verbose = true);
println(length(unique(Z)))

Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
Louvain Iteration 5
Louvain Iteration 6
Louvain Iteration 7
Louvain Iteration 8
Louvain Iteration 9
Louvain Iteration 10
Louvain Iteration 11
Louvain Iteration 12
5


In [97]:
Profile.print(sortedby = :count, mincount = 100)

Overhead ╎ [+additional indent] Count File:Line; Function
    ╎8329  @Base/task.jl:358; (::IJulia.var"#15#18")()
    ╎ 8329  ...ia/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
    ╎  8329  @Base/essentials.jl:711; invokelatest
    ╎   8329  @Base/essentials.jl:712; #invokelatest#1
    ╎    8329  ...execute_request.jl:67; execute_request(::ZMQ.Socket...
    ╎     8329  ...oftGlobalScope.jl:218; softscope_include_string(::...
   8╎    ╎ 8329  @Base/boot.jl:331; eval
    ╎    ╎  8320  ...rgraph_louvain.jl:2; (::HypergraphModularity.var...
   3╎    ╎   8320  ...rgraph_louvain.jl:2; HyperLouvain##kw
  21╎    ╎    976   ...raph_louvain.jl:74; HyperLouvain(::hypergraph...
    ╎    ╎     955   @Base/reduce.jl:503; sum(::Base.Generator{Bas...
    ╎    ╎    ╎ 955   @Base/reduce.jl:486; sum
    ╎    ╎    ╎  955   @Base/reduce.jl:283; mapreduce
    ╎    ╎    ╎   955   @Base/reduce.jl:283; #mapreduce#193
    ╎    ╎    ╎    955   @Base/reduce.jl:157; mapfoldl
    ╎    ╎    ╎     955   @Base/reduce.jl